In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
year=2024
data=pd.read_csv("BestPictureNominations.csv")
data=data.drop(labels=['Win', 'SAG', 'WGA', 'DGA', 'PGA', 'BAFTA', 'OscarNoms', 'DirectingNom', 'ActingNom', 'ScreenplayNom', 'EditingNom'], axis=1)
#data=data[data['Year']!=2023]
train=data[data['Year']!=year]
y_train=train['Best Picture Nominee']
X_train=train.drop(labels=['Best Picture Nominee', 'Title', "Year"], axis=1)

test=data[data['Year']==year]
y_test=test['Best Picture Nominee']
X_test_titles=test['Title']
X_test=test.drop(labels=['Best Picture Nominee', "Title", "Year"], axis=1)

scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32).reshape(-1, 1)

X_test_1 = torch.tensor(X_test, dtype=torch.float32)
y_test_1 = torch.tensor(np.array(y_test), dtype=torch.float32).reshape(-1, 1)

In [3]:
model_1 = nn.Sequential(
    nn.Linear(22, 28),
    nn.ReLU(),
    nn.Linear(28, 19),
    nn.ReLU(),
    nn.Linear(19, 1),
    nn.Sigmoid()
)
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model_1.parameters(), lr=0.001)

n_epochs = 100
batch_size = 10
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model_1(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #print(f'Finished epoch {epoch}, latest loss {loss}')

In [4]:
data=pd.read_csv("BestPictureNominations.csv")
#data=data[data['Best Picture Nominee']==1]
#data=data[data['Year']!=2023]
train=data[data['Year']!=year]


y_train=train['Win']
X_train=train.drop(labels=['Best Picture Nominee', 'Title', "Year", 'Win'], axis=1)

test=data[data['Year']==year]
y_test=test['Win']
X_test_titles=test['Title']
X_test=test.drop(labels=['Best Picture Nominee', "Title", "Year", "Win"], axis=1)

scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32).reshape(-1, 1)

X_test_2 = torch.tensor(X_test, dtype=torch.float32)
y_test_2 = torch.tensor(np.array(y_test), dtype=torch.float32).reshape(-1, 1)

In [5]:
model_2 = nn.Sequential(
    nn.Linear(32, 38),
    nn.ReLU(),
    nn.Linear(38, 19),
    nn.ReLU(),
    nn.Linear(19, 1),
    nn.Sigmoid()
)
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model_2.parameters(), lr=0.001)

In [6]:
n_epochs = 100
batch_size = 10
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model_2(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #print(f'Finished epoch {epoch}, latest loss {loss}')

In [7]:
with torch.no_grad():
    y_pred_1 = model_1(X_test_1)
with torch.no_grad():
    y_pred_2 = model_2(X_test_2)

#for i in range(len(y_pred_1)):
#    if i<10:
#        if y_pred_1[i]>0.5:
#            y_pred_1[i]=1
#        else:
#            y_pred_1[i]=0
#    else:
#        y_pred_1[i]=0
 
accuracy = (y_pred_1.round() == y_test_1).float().mean()
print(f"Nomination Accuracy: {accuracy}")

output=pd.DataFrame()
y_pred_1 = [float(i) for i in y_pred_1]
y_pred_2 = [float(i/sum(y_pred_2)) for i in y_pred_2]
output['Title']=X_test_titles
output['Probability Nom']=y_pred_1
output['Probability Win']=y_pred_2
output.sort_values(by=['Probability Nom'], axis=0, ascending=False)

Nomination Accuracy: 0.1428571492433548


,Title,Probability Nom,Probability Win
0,Anora,0.999993,0.024929
2,Conclave,0.998546,0.054433
13,The Room Next Door,0.998342,0.053880
6,Dune: Part Two,0.997452,0.015688
7,Gladiator II,0.992185,0.013320
11,A Complete Unknown,0.990840,0.032357
5,Blitz,0.990840,0.032357
17,Hard Truths,0.982548,0.019408
18,Challengers,0.981392,0.020005
3,The Brutalist,0.877286,0.094147
